In [1]:
import json
from datetime import datetime
import pandas as pd
import os
import win32com.client
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font, Border, Side

import warnings

warnings.simplefilter(action="ignore", category=Warning)

In [2]:
# Номер акта исследования по которому делался предыдущий анализ
user_number = 927
# user_number = int(input("Введите номер акта исследования: "))

In [3]:
year_now = datetime.today().year  # текущий год

# имя файла базы рекламаций ОТК с учетом текущего года
# file = "//Server/otk/1 ГАРАНТИЯ на сервере/" + str(year_now) + "-2019_ЖУРНАЛ УЧЁТА.xlsm"
file = f"D:/РАБОТА/{str(year_now)}-2019_ЖУРНАЛ УЧЁТА.xlsm"

In [4]:
# ------------------------------- Создаем датафрейм из файла Excel базы ОТК -----------------------------------
df = pd.read_excel(
    file,
    sheet_name=str(year_now),
    usecols=[
        "Период выявления дефекта (отказа)",
        "Обозначение изделия",
        "Заводской номер изделия",
        "Дата изготовления изделия",
        "Количество предъявленных изделий",
        "Номер акта исследования",
        "Виновник дефекта - БЗА",
        "Виновное подразделение",
        "Причины возникновения дефектов",
        "Пояснения к причинам возникновения дефектов",
    ],
    header=1,
)
# header=1 - строку 2 таблицы ОТК делаем заголовками столбцов датафрейма (индексы строк начинаются с 0)

In [5]:
# ---------------------- Обработка отсутствующих значений и типов данных -------------------

# Удаляем строки с отсутствующим номером акта исследования
df.dropna(subset=['Номер акта исследования'], inplace=True)

# Заменяем NaN на "БЗА" только если в столбце "Виновник дефекта - БЗА" стоит "+"
# вариант 1 - одним выражением
df.loc[
    (df['Виновное подразделение'].isna()) & (df['Виновник дефекта - БЗА'] == '+'),
    'Виновное подразделение'
] = "БЗА"

# вариант 2 - более читаемый через отдельное условие и замену
# # Условие: NaN в "Виновное подразделение" И "+" в "Виновник дефекта - БЗА"
# condition = (
#     df['Виновное подразделение'].isna() &
#     (df['Виновник дефекта - БЗА'] == '+')
# )
# df.loc[condition, 'Виновное подразделение'] = "БЗА"

# Изменяем тип данных с float на int
df["Количество предъявленных изделий"] = df["Количество предъявленных изделий"].astype("int32")

# Извлекаем номер акта исследования как строку (сохраняем суффиксы)
df['Номер акта исследования'] = df['Номер акта исследования'].str.split(' № ').str[1]

In [6]:
# ------------------------ Вспомогательные функции и манипуляции --------------------------

# Функция для безопасного извлечения числовой части для сравнения
def get_numeric_part(act_str):
    """Извлекает числовую часть для сравнения: '824-1' -> 824"""
    try:
        return int(act_str.split('-')[0])
    except (ValueError, IndexError, AttributeError):
        return 0

# Создаем столбец для сравнения
df['act_number'] = df['Номер акта исследования'].apply(get_numeric_part)

In [7]:
# ------------------------ Основная логика и вывод результата ------------------------------

# Фильтрация по числовой части (оставляем акты с номером начиная с user_number + 1)
df_filtered = df[df['act_number'] > (user_number)]

# Извлекаем номера актов
act_numbers = df_filtered['Номер акта исследования'].unique()

# Сортируем по кастомному ключу и берем последний
max_act_number = sorted(act_numbers, key=lambda x: (
    int(x.split('-')[0]) if '-' in x else int(x),
    int(x.split('-')[1]) if '-' in x else 0
))[-1]

# print(f"Максимальный номер акта исследования: {max_act_number}")

# Убираем служебный столбец
df_filtered = df_filtered.drop(columns='act_number')

# Группировка
df_grouped = df_filtered.groupby([
    'Виновное подразделение',
    'Период выявления дефекта (отказа)',
    'Обозначение изделия'
]).agg({
    'Заводской номер изделия': lambda x: ', '.join(x.dropna().astype(str).unique()),
    'Дата изготовления изделия': lambda x: ', '.join(x.dropna().unique()),
    'Количество предъявленных изделий': 'sum',
    'Номер акта исследования': lambda x: ', '.join(x.dropna().unique()),
    'Причины возникновения дефектов': lambda x: ', '.join(x.dropna().unique()),
    'Пояснения к причинам возникновения дефектов': lambda x: ', '.join(x.dropna().unique())
})

# ------------------- Разделяем по мультииндексу -----------------------

# Датафрейм с неопределенным виновником (Виновник БЗА)
df_bza = df_grouped.loc[df_grouped.index.get_level_values('Виновное подразделение') == 'БЗА']
# print(df_bza)

# Датафрейм с виновниками
df_not_bza = df_grouped.loc[df_grouped.index.get_level_values('Виновное подразделение') != 'БЗА']
# print(df_not_bza)

In [8]:
print(f"Максимальный номер акта исследования: {max_act_number}")

Максимальный номер акта исследования: 1038


In [9]:
df_bza

Заводской номер изделия  \
Виновное подразделение Период выявления дефекта (отказа) Обозначение изделия                                                   
БЗА                    Гомсельмаш - эксплуатация         5336-3509012-05                           2338, 1442, 44, 2271, 839   
                                                         5336-3509012-10                                    4936, 5444, 5300   
                       ММЗ - АСП                         245-1307010-А1-11                                                     
                                                         3LD-1307010-Б                                                         
                                                         А29.05.000А-03-БЗА                                            11177   
                       ММЗ - эксплуатация                240-1307010-А1                                                        
                                                         245-1307010-А1-11                                                     
                                                         245-1307010-Г                                                         
                                                         246.9-1307010                                                         
                                                         260-1307116-02                                                        
                                                         260-1307116-04                                                        
                                                         260-1307116-05                                                        
                                                         260-1307116-06                                                        
                                                         260-1307116-15                                                        
                                                         260-1307117-02                                                        
                                                         А29.05.000-03-БЗА                                             36232   
                                                         А29.05.000А-03-БЗА                                            30719   
                                                         А29.05.000А-БЗА                                               28498   
                       Ростсельмаш - АСП                 5336-3509012                       11534, 11545, 11922, 7371, 12149   
                       Ростсельмаш - эксплуатация        5336-3509012-К181                                         967, 3934   
                       ЯМЗ - эксплуатация                КБПА 451363.21 \n(536.1307010-21)                                     
                                                         КБПА 451363.41 \n(536.1307010-41)                                     
                       запчасть                          260-1307116-06                                                        
                                                         260-1307116-15                                                        

                                                                                                                    Дата изготовления изделия  \
Виновное подразделение Период выявления дефекта (отказа) Обозначение изделия                                                                    
БЗА                    Гомсельмаш - эксплуатация         5336-3509012-05                                           12.22, 07.22, 01.23, 02.23   
                                                         5336-3509012-10                                                                07.24   
                       ММЗ - АСП                         245-1307010-А1-11                                                                      
                                               

In [10]:
df_not_bza

Заводской номер изделия  \
Виновное подразделение Период выявления дефекта (отказа) Обозначение изделия                              
2-3-0                  ММЗ - АСП                         50-1007210-А4                                    
2-7-1                  ММЗ - эксплуатация                А29.05.000А-БЗА      1817, 44701, 28385, 45625   
3-7-0                  ММЗ - АСП                         245-1307010-А1-11                                
3-9-1                  ММЗ - эксплуатация                ТКР 7-02.08                               2470   
                       запчасть                          ТКР 7-00.01                               1992   
ОЗ                     Гомсельмаш - эксплуатация         5336-3509012-10                           3958   
                       ММЗ - АСП                         245-1017015-Б                                    
                                                         246.1М-1307010                                   
                       ММЗ - эксплуатация                245-1307010-А1-11                                
                       запчасть                          240-1307010-А1                                   
                                                         260-1307116-02                                   
УМС                    ММЗ - АСП                         3LD-1307010-Б                                    

                                                                             Дата изготовления изделия  \
Виновное подразделение Период выявления дефекта (отказа) Обозначение изделия                             
2-3-0                  ММЗ - АСП                         50-1007210-А4                           07.25   
2-7-1                  ММЗ - эксплуатация                А29.05.000А-БЗА           01.23, 11.23, 08.23   
3-7-0                  ММЗ - АСП                         245-1307010-А1-11                       06.25   
3-9-1                  ММЗ - эксплуатация                ТКР 7-02.08                             10.23   
                       запчасть                          ТКР 7-00.01                            06.025   
ОЗ                     Гомсельмаш - эксплуатация         5336-3509012-10                         10.23   
                       ММЗ - АСП                         245-1017015-Б                           05.25   
                                                         246.1М-1307010                          03.25   
                       ММЗ - эксплуатация                245-1307010-А1-11                               
                       запчасть                          240-1307010-А1                   11.23, 02.25   
                                                         260-1307116-02            04.__, 12.24, 02.25   
УМС                    ММЗ - АСП                         3LD-1307010-Б                           04.25   

                                                                              Количество предъявленных изделий  \
Виновное подразделение Период выявления дефекта (отказа) Обозначение изделия                                     
2-3-0                  ММЗ - АСП                         50-1007210-А4                                      32   
2-7-1                  ММЗ - эксплуатация                А29.05.000А-БЗА                                     4   
3-7-0                  ММЗ - АСП                         245-1307010-А1-11                                   1   
3-9-1                  ММЗ - эксплуатация                ТКР 7-02.08                                         1   
                       запчасть                          ТКР 7-00.01                                         1   
ОЗ                     Гомсельмаш - эксплуатация         5336-3509012-10                                     1   
                       ММЗ - АСП                         245-1017015-Б                                       1   
                                                         246.1М-130701